# Análise Exploratória dos Dados da SPTrans

**Objetivo:** Este notebook é o nosso ambiente de análise para explorar os dados coletados pela nossa pipeline e extrair os primeiros insights.

### 1. Importação das Bibliotecas e Conexão com o Banco de Dados

In [6]:
import pandas as pd
import sqlite3
import os

# Define o caminho para o banco de dados
DB_PATH = os.path.join('..', 'data', 'sptrans_data.db')

# Cria a conexão com o banco
conn = sqlite3.connect(DB_PATH)

print(f"Conectado ao banco de dados em: {DB_PATH}")

Conectado ao banco de dados em: ../data/sptrans_data.db


### 2. Carregar os Dados da Análise

Vamos carregar a tabela `resultados_analise`, que contém os dados enriquecidos que nosso script `analise_onibus.py` gerou. Filtramos para pegar apenas os registros que possuem dados de previsão.

In [ ]:
query = "SELECT * FROM resultados_analise WHERE horario_previsto_chegada IS NOT NULL;"

df = pd.read_sql_query(query, conn)

# Exibe as 5 primeiras linhas para inspecionar os dados
df.head()

### 3. O Primeiro Insight: Cálculo do Atraso

Esta é a etapa mais importante. Vamos calcular a diferença entre a previsão de chegada e o horário real da posição do ônibus para encontrar nosso indicador de atraso.

**Nota:** Um valor positivo significa que o ônibus está ADIANTADO (passou antes da previsão). Um valor negativo significa que está ATRASADO.

In [ ]:
# Passo 1: Converter a coluna de posição para datetime e JÁ REMOVER o fuso horário.
    # Esta é a correção principal. Fazemos isso primeiro.
df['horario_posicao'] = pd.to_datetime(df['horario_posicao']).dt.tz_localize(None)
    
    # Passo 2: Agora que a coluna não tem mais fuso, podemos extrair a data com segurança.
df['data_coleta'] = df['horario_posicao'].dt.date
    
   # Passo 3: Combinar a data com o horário previsto para criar uma coluna de datetime completa.
df['horario_previsto_chegada_dt'] = pd.to_datetime(df['data_coleta'].astype(str) + ' ' + df[
      'horario_previsto_chegada'])
    
   # Passo 4: Calcular a diferença em minutos. Agora isso vai funcionar, pois as duas colunas são "ingênuas".
   # Atraso = Hora Real - Hora Prevista
df['atraso_minutos'] = (df['horario_posicao'] - df['horario_previsto_chegada_dt']).dt.total_seconds() / 60
    
    # Passo 5: Exibir o resultado.
print("Cálculo de atraso concluído! Valores positivos = Atrasado, Valores negativos = Adiantado.")
df[['id_onibus', 'letreiro_linha', 'horario_previsto_chegada_dt', 'horario_posicao', 'atraso_minutos']].head()


In [ ]:
# Converte as colunas de texto para o formato de data e hora (datetime)
# O formato da API para horario_posicao é ISO 8601 (ex: '2025-08-15T22:07:05Z')
df['horario_posicao'] = pd.to_datetime(df['horario_posicao']).dt.tz_localize(None)

#O formato para horario_previsto_chegada é apenas HH:MM. Precisamos combiná-lo com a data da coleta.
# Primeiro, extraímos a data da coluna horario_posicao
df['data_coleta'] = df['horario_posicao'].dt.date
# Agora, combinamos a data com o horário previsto
df['horario_previsto_chegada_dt'] = pd.to_datetime(df['data_coleta'].astype(str) + ' ' + df['horario_previsto_chegada'])

# Calcula a diferença em minutos
df['atraso_minutos'] = (df['horario_previsto_chegada_dt'] - df['horario_posicao']).dt.total_seconds() / 60

# Exibe o resultado com a nova coluna de atraso
df[['id_onibus', 'letreiro_linha', 'horario_previsto_chegada_dt', 'horario_posicao', 'atraso_minutos']].head()

### Próximos Passos

Agora que temos o atraso calculado, podemos começar a fazer perguntas mais interessantes:

- Qual a média de atraso por linha?
- Qual linha tem o maior atraso?
- O atraso varia com a hora do dia?

In [ ]:
import matplotlib.pyplot as plt
     
     # Calcula o atraso médio por linha
atraso_por_linha = df.groupby('letreiro_linha')['atraso_minutos'].mean().sort_values()
     
     # Cria o gráfico de barras horizontais
atraso_por_linha.plot(kind='barh', figsize=(10, 8))
plt.xlabel('Atraso Médio (Minutos)')
plt.ylabel('Linha de Ônibus')
plt.title('Atraso Médio por Linha de Ônibus')
plt.grid(axis='x', linestyle='--')
plt.show()

